In [113]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [114]:
## Other imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import (
  accuracy_score,
  precision_score,
  recall_score,
  f1_score,
  ConfusionMatrixDisplay,
)

In [115]:
# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device is {device}")

In [116]:
## Data loading

def load_data(path):
    df = pd.read_csv(path)
    X = df.drop("label", axis=1).to_numpy()
    Y = df["label"].to_numpy()
    return X, Y

# Training data
X, Y = load_data("/kaggle/input/digit-recognizer/train.csv")

# Test data
X_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv").to_numpy()

In [117]:
## Data exploration

# Plot the number of samples per class
labels, counts = np.unique(Y, return_counts=True)
plt.bar([str(label) for label in labels], counts)
plt.xlabel("Class")
plt.ylabel("Number of samples")
plt.title("Number of samples per class")

# Plot one sample from each class
fig = plt.figure(figsize=[20, 8])
for label in labels:
    first_label_idx = np.where(Y == label)[0][0]
    ax = fig.add_subplot(2 , 5, label+1)
    ax.imshow(X[first_label_idx].reshape((28,28)), cmap="gray")
    ax.set_title(f"Sample of class {label}")

In [118]:
## Split training data into train and validation sets

def split_train_validation(data_length, percentage=0.8):
    shuffled_indices = np.random.permutation(data_length)
    n_train = round(data_length * percentage)
    idxs_train = shuffled_indices[:n_train]
    idxs_validation = shuffled_indices[n_train:]
    return idxs_train, idxs_validation

idxs_train, idxs_validation = split_train_validation(len(Y))
X_train, Y_train = X[idxs_train], Y[idxs_train]
X_validation, Y_validation = X[idxs_validation], Y[idxs_validation]

In [123]:
## Create PyTorch datasets

class DigitDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform
    
    def __getitem__(self, idx):
        x = self.X[idx].reshape(28,28)
        y = self.Y[idx]
        
        if self.transform:
            x = Image.fromarray(x.astype(np.uint8))
            x = self.transform(x)
        
        return x, y
    
    def __len__(self):
        return len(self.X)

transform = transforms.ToTensor()
train_dataset = DigitDataset(X_train, Y_train, transform)
validation_dataset = DigitDataset(X_validation, Y_validation, transform)
combined_dataset = DigitDataset(X, Y, transform)

In [120]:
## LeNet inspired network

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.feature_extraction_layers = nn.Sequential( # Input dim = (batch_size, 1, 28, 28)
            nn.Conv2d(1, 6, kernel_size=5), # Output dim = (batch_size, 6, 24, 24)
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(2), # Output dim = (batch_size, 6, 12, 12)
            nn.Conv2d(6, 16, kernel_size=5), # Output dim = (batch_size, 16, 8, 8)
            nn.BatchNorm2d(16),
            nn.ReLU(),
        )
        
        self.classification_layer = nn.Sequential(
            nn.Linear(1024, 600),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(600, 100),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(100, 10),
            nn.Softmax(dim=1),
        )
        
    def forward(self, x):
        x = self.feature_extraction_layers(x)
        x = torch.flatten(x, 1)
        x = self.classification_layer(x)
        return x

In [140]:
## Network training

def train_model(model, criterion, optimizer, batch_size, dataset):
    dataloader = DataLoader(dataset=train_dataset, batch_size=train_batch_size)
    
    for epoch in tqdm(range(epochs)):
        model.train()
        loss_curve = []

        for X, Y in dataloader:
            optimizer.zero_grad()

            X = X.to(device)
            Y = Y.to(device)

            # Forward pass
            Y_predicted = model(X)
            loss = criterion(Y_predicted, Y)
            loss_curve += [loss.item()]

            # Backward pass
            loss.backward()
            optimizer.step()

        print(f"--- Iteration {epoch+1}: training loss = {np.array(loss_curve).mean():.4f} ---")

# Define the parameters
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

train_batch_size = 100
epochs = 20

train_model(model, criterion, optimizer, train_batch_size, train_dataset)

In [141]:
## Deploy model on validation set

validation_dataloader = DataLoader(dataset=validation_dataset)

predicted = []
actual = []
with torch.no_grad():
    model.eval()
    
    for x, y in tqdm(validation_dataloader):
        x = x.to(device)
        y = y.detach().numpy()
        
        label_probabilities = model(x).cpu().detach().numpy()
        label_predicted = np.argmax(label_probabilities)
        
        predicted.append(label_predicted)
        actual.append(y)

predicted = np.array(predicted)
actual = np.array(actual)

In [142]:
## Evaluate on evaluation set

# Compute performance metrics
print(f"Accuracy: { accuracy_score(actual, predicted)}")
print(f"Precision scores (index corresponds to class): {precision_score(actual, predicted, average=None)}")
print(f"Recall scores (index corresponds to class): {recall_score(actual, predicted, average=None)}")
print(f"F1 scores (index corresponds to class): {f1_score(actual, predicted, average=None)}")

# Visualize the confusion matrix
cmp = ConfusionMatrixDisplay.from_predictions(actual, predicted)
_, ax = plt.subplots(figsize=(10,10))
cmp.plot(ax=ax)

In [143]:
## Train on combined training and validation data + deploy on test data

train_model(model, criterion, optimizer, train_batch_size, combined_dataset)

# Deploy on test data
predicted = []
with torch.no_grad():
    model.eval()
    
    for x in tqdm(X_test):
        x = x.reshape(28,28)
        x = Image.fromarray(x.astype(np.uint8))
        x = transform(x).unsqueeze(0).to(device)
        
        label_probabilities = model(x).cpu().detach().numpy()
        label_predicted = np.argmax(label_probabilities)
        
        predicted.append(label_predicted)

# Save test predictions for submission
labels = pd.Series(predicted, name="Label")
submission = pd.concat([pd.Series(range(1,len(X_test)+1), name="ImageId"), labels], axis=1)
submission.to_csv("digit_recognizer_submission.csv", index=False)